# What was the impact of natural disasters in risk pool regions?

Data Source: EM-DAT: The Emergency Events Database - Université catholique de Louvain (UCL) - CRED, D. Guha-Sapir - https://www.emdat.be, Brussels, Belgium.

Downloaded on: 2018-11-22

FAQ for the database: https://www.emdat.be/frequently-asked-questions

Time frame: since risk pool establishment, so different time frames for each risk pool

Disasters that are included in this analysis: Out of all recorded natural disasters in the EMDAT database, this analysis looks at a subset of data which only includes natural disasters of the following categories: Riverine flood, Flash flood, Coastal flood, Tropical cyclone, Extra-tropical storm, Heat wave, Drought, Ground movement, Tsunami

Earthquakes (tsunamis and ground movements) were excluded later in the analysis

Damage cost is not available for all disasters recorded in the database, so value sums are MINIMUM damage (real ones higher)

Unit of damage cost: "The estimated damage is given in US $ (in thousands) in the value of the year of occurrence."

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

import plotly
import plotly.express as px
import plotly.graph_objects as go
import cufflinks as cf
import plotly.offline as pyo
import plotly.figure_factory as ff
from plotly.offline import init_notebook_mode,plot,iplot
import folium
pyo.init_notebook_mode(connected=True)
cf.go_offline()

In [2]:
df=pd.read_csv('data\EMDAT\EMDAT-climateinsurance-ev.csv',delimiter=';', encoding=('latin-1'))
df

,year,iso,country_name,continent,region,disaster_subtype,disaster_type,disaster_subgroup,occurrence,Total_deaths,Injured,Affected,Homeless,Total_affected,Total_damage_K_USD
0,1998,AUS,Australia,Oceania,Australia and New Zealand,Coastal flood,Flood,Hydrological,2,4.0,NaN,1375.0,NaN,1375.0,161300.0
1,1998,JPN,Japan,Asia,Eastern Asia,Coastal flood,Flood,Hydrological,1,16.0,33.0,24000.0,NaN,24033.0,NaN
2,1998,MEX,Mexico,Americas,Central America,Coastal flood,Flood,Hydrological,1,274.0,NaN,25000.0,NaN,25000.0,602700.0
3,1999,KHM,Cambodia,Asia,South-Eastern Asia,Coastal flood,Flood,Hydrological,1,NaN,NaN,106670.0,17805.0,124475.0,NaN
4,1999,COL,Colombia,Americas,South America,Coastal flood,Flood,Hydrological,1,3.0,NaN,NaN,1100.0,1100.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3120,2011,JPN,Japan,Asia,Eastern Asia,Tsunami,Earthquake,Geophysical,1,19846.0,5933.0,362887.0,NaN,368820.0,210000000.0
3121,2013,SLB,Solomon Islands,Oceania,Melanesia,Tsunami,Earthquake,Geophysical,1,10.0,NaN,3329.0,NaN,3329.0,2000.0
3122,2015,CHL,Chile,Americas,South America,Tsunami,Earthquake,Geophysical,1,19.0,15.0,681484.0,NaN,681499.0,800000.0
3123,2016,NZL,New Zealand,Oceania,Australia and New Zealand,Tsunami,Earthquake,Geophysical,1,2.0,50.0,NaN,NaN,50.0,3900000.0


In [3]:
disaster_subtype=df['disaster_subtype'].value_counts()
country_name=df['country_name'].value_counts()
continent=df['continent'].value_counts()
year=df['year'].value_counts()
disaster_subtype

Riverine flood          1191
Tropical cyclone         543
Flash flood              413
Drought                  347
Ground movement          318
Heat wave                132
Extra-tropical storm     105
Coastal flood             46
Tsunami                   30
Name: disaster_subtype, dtype: int64

In [4]:
px.histogram(df, x='disaster_subtype')


In [5]:
px.histogram(df, x='disaster_type')

In [6]:
px.histogram(df, x='continent')

In [22]:
fig=px.histogram(df, x='year')
fig.update_layout(bargap=0.5,xaxis = dict(tickangle =45,tick0 =0, dtick =1),yaxis=dict(dtick=20))

In [23]:
px.histogram(df, x='continent',y='Total_deaths',col)

In [9]:
# creating groups of countries that have received a payout (_recipients) 
# and that would have not received a payout (_potentials)

Americas_recipients = ['Anguilla','Antigua & Barbuda','Bahamas','Barbados','Belize','Dominica','St. Kitts & Nevis','Saint Lucia','St. Vincent & the Grenadines','Trinidad & Tobago','Turks & Caicos Islands','Nicaragua','Haiti']
Americas_potentials = ['Bermuda','Cayman Islands','Grenada','Jamaica']
Americas_all = Americas_recipients + Americas_potentials


Oceania_recipients = ['Tonga', 'Vanuatu']
Oceania_potentials = ['Cook Islands','Marshall Islands','Samoa','Fiji','Kiribati','Micronesia','Nauru','Niue','Palau','Papua New Guinea','Solomon Islands','Tuvalu']
Oceania_all = Oceania_recipients + Oceania_potentials


Africa_recipients = ['Malawi','Mauritania','Niger','Senegal']
Africa_potentials = ['Gambia','Kenya','Mali']
Africa_optionals = ['Benin','Burkina Faso','Burundi','CAR','Chad','Comoros','Republic Congo','Djibouti','Gabon','Guinea','Guinea-Bissau','Ivory Coast','Liberia','Libya','Madagascar','Mozambique','Nigeria','Rwanda','Sahrawi','Sao Tome & Principe','Sierra Leone','Togo','Zambia','Zimbabwe']
Africa_all = Africa_recipients + Africa_potentials

In [10]:
# make dfs that are policy for each risk pool as in regions covered and timepoint of establishment

Americas_est = 2007
Africa_est = 2014
Oceania_est = 2016

df_Americas = pd.DataFrame()
df_Oceania = pd.DataFrame()
df_Africa = pd.DataFrame()

pools = [Americas_all, Africa_all, Oceania_all]

for risk_pool in pools:
    if 'Anguilla' in risk_pool:
        df_time_temp = df[df['year']>=Americas_est]
        for country in risk_pool:
            df_temp = df_time_temp[df_time_temp['country_name']==country]
            df_Americas = df_Americas.append(df_temp)
    if 'Tonga' in risk_pool:
        df_time_temp = df[df['year']>=Oceania_est]
        for country in risk_pool:
            df_temp = df_time_temp[df_time_temp['country_name']==country]
            df_Oceania = df_Oceania.append(df_temp)
    if 'Malawi' in risk_pool:
        df_time_temp = df[df['year']>=Africa_est]
        for country in risk_pool:
            df_temp = df_time_temp[df_time_temp['country_name']==country]
            df_Africa = df_Africa.append(df_temp)            
            
risk_pool_dfs = pd.concat([df_Americas,df_Oceania,df_Africa])
risk_pool_dfs

,year,iso,country_name,continent,region,disaster_subtype,disaster_type,disaster_subgroup,occurrence,Total_deaths,Injured,Affected,Homeless,Total_affected,Total_damage_K_USD
3029,2017,AIA,Anguilla,Americas,Caribbean,Tropical cyclone,Storm,Meteorological,1,4.0,NaN,15000.0,NaN,15000.0,200000.0
253,2010,BRB,Barbados,Americas,Caribbean,Drought,Drought,Climatological,1,NaN,NaN,NaN,NaN,NaN,NaN
1061,2007,BRB,Barbados,Americas,Caribbean,Ground movement,Earthquake,Geophysical,1,NaN,1.0,NaN,NaN,1.0,NaN
2876,2010,BRB,Barbados,Americas,Caribbean,Tropical cyclone,Storm,Meteorological,1,NaN,NaN,2500.0,NaN,2500.0,NaN
3034,2017,BRB,Barbados,Americas,Caribbean,Tropical cyclone,Storm,Meteorological,1,1.0,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
836,2015,KEN,Kenya,Africa,Eastern Africa,Flash flood,Flood,Hydrological,1,13.0,NaN,1500.0,NaN,1500.0,NaN
2477,2015,KEN,Kenya,Africa,Eastern Africa,Riverine flood,Flood,Hydrological,2,125.0,73.0,243726.0,NaN,243799.0,NaN
2515,2016,KEN,Kenya,Africa,Eastern Africa,Riverine flood,Flood,Hydrological,1,3.0,NaN,NaN,1000.0,1000.0,NaN
2539,2017,KEN,Kenya,Africa,Eastern Africa,Riverine flood,Flood,Hydrological,1,26.0,NaN,25000.0,NaN,25000.0,NaN


# how many people have been affected in the past by natural disasters in the risk pool regions

In [11]:
px.histogram(risk_pool_dfs,x='continent',y='Affected')

# how many disasters happened in that time in that risk pool region

In [12]:
px.histogram(risk_pool_dfs, y='occurrence',x='continent')

# how much damage did disasters cause in the risk pool regions since establishment of the respective pool

In [13]:
px.histogram(risk_pool_dfs, y='Total_damage_K_USD',x='continent')

## only look at disasters insured under each risk pool

In [14]:
Americas_disaster_types = ['Tropical cyclone', 'Riverine flood', 'Flash flood', 'Ground movement', 'Tsunami']
Oceania_disaster_types = ['Tropical cyclone']
Africa_disaster_types = ['Drought']


df_Americas_policy = pd.DataFrame()
df_Oceania_policy = pd.DataFrame()
df_Africa_policy = pd.DataFrame()

pools = [Americas_all, Africa_all, Oceania_all]

for risk_pool in pools:
    if 'Anguilla' in risk_pool:
        df_time_temp2 = df[df['year']>=Americas_est]
        for country in risk_pool:
            df_temp2 = df_time_temp2[df_time_temp2['country_name']==country]
            for disaster_type in Americas_disaster_types:
                df_type_temp = df_temp2[df_temp2['disaster_subtype']==disaster_type]
                df_Americas_policy = df_Americas_policy.append(df_type_temp)

    if 'Tonga' in risk_pool:
        df_time_temp2 = df[df['year']>=Oceania_est]
        for country in risk_pool:
            df_temp2 = df_time_temp2[df_time_temp2['country_name']==country]
            for disaster_type in Oceania_disaster_types:
                df_type_temp = df_temp2[df_temp2['disaster_subtype']==disaster_type]
                df_Oceania_policy = df_Oceania_policy.append(df_type_temp)
                
    if 'Malawi' in risk_pool:
        df_time_temp2 = df[df['year']>=Africa_est]
        for country in risk_pool:
            df_temp2 = df_time_temp2[df_time_temp2['country_name']==country]
            for disaster_type in Africa_disaster_types:
                df_type_temp = df_temp2[df_temp2['disaster_subtype']==disaster_type]
                df_Africa_policy = df_Africa_policy.append(df_type_temp)  
                
risk_pool_policy = pd.concat([df_Americas_policy,df_Oceania_policy,df_Africa_policy])
risk_pool_policy                

,year,iso,country_name,continent,region,disaster_subtype,disaster_type,disaster_subgroup,occurrence,Total_deaths,Injured,Affected,Homeless,Total_affected,Total_damage_K_USD
3029,2017,AIA,Anguilla,Americas,Caribbean,Tropical cyclone,Storm,Meteorological,1,4.0,NaN,15000.0,NaN,15000.0,200000.0
2876,2010,BRB,Barbados,Americas,Caribbean,Tropical cyclone,Storm,Meteorological,1,NaN,NaN,2500.0,NaN,2500.0,NaN
3034,2017,BRB,Barbados,Americas,Caribbean,Tropical cyclone,Storm,Meteorological,1,1.0,NaN,NaN,NaN,NaN,NaN
1061,2007,BRB,Barbados,Americas,Caribbean,Ground movement,Earthquake,Geophysical,1,NaN,1.0,NaN,NaN,1.0,NaN
2791,2007,BLZ,Belize,Americas,Central America,Tropical cyclone,Storm,Meteorological,1,NaN,NaN,20000.0,NaN,20000.0,14847.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
344,2015,MWI,Malawi,Africa,Eastern Africa,Drought,Drought,Climatological,1,NaN,NaN,6700000.0,NaN,6700000.0,NaN
379,2017,MRT,Mauritania,Africa,Western Africa,Drought,Drought,Climatological,1,NaN,NaN,3893774.0,NaN,3893774.0,NaN
327,2014,SEN,Senegal,Africa,Western Africa,Drought,Drought,Climatological,1,NaN,NaN,639702.0,NaN,639702.0,NaN
323,2014,KEN,Kenya,Africa,Eastern Africa,Drought,Drought,Climatological,1,NaN,NaN,1600000.0,NaN,1600000.0,NaN


In [15]:
px.histogram(risk_pool_policy, y='Total_damage_K_USD',x='continent')

## Repeat first analysis, without earthquake data 

In [16]:
df_woe = df[df['disaster_type']!= 'Earthquake']
df_woe

,year,iso,country_name,continent,region,disaster_subtype,disaster_type,disaster_subgroup,occurrence,Total_deaths,Injured,Affected,Homeless,Total_affected,Total_damage_K_USD
0,1998,AUS,Australia,Oceania,Australia and New Zealand,Coastal flood,Flood,Hydrological,2,4.0,NaN,1375.0,NaN,1375.0,161300.0
1,1998,JPN,Japan,Asia,Eastern Asia,Coastal flood,Flood,Hydrological,1,16.0,33.0,24000.0,NaN,24033.0,NaN
2,1998,MEX,Mexico,Americas,Central America,Coastal flood,Flood,Hydrological,1,274.0,NaN,25000.0,NaN,25000.0,602700.0
3,1999,KHM,Cambodia,Asia,South-Eastern Asia,Coastal flood,Flood,Hydrological,1,NaN,NaN,106670.0,17805.0,124475.0,NaN
4,1999,COL,Colombia,Americas,South America,Coastal flood,Flood,Hydrological,1,3.0,NaN,NaN,1100.0,1100.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3090,2018,TWN,Taiwan (Province of China),Asia,Eastern Asia,Tropical cyclone,Storm,Meteorological,1,7.0,140.0,6000.0,NaN,6140.0,34000.0
3091,2018,TON,Tonga,Oceania,Polynesia,Tropical cyclone,Storm,Meteorological,1,NaN,NaN,87000.0,NaN,87000.0,NaN
3092,2018,USA,United States of America (the),Americas,Northern America,Tropical cyclone,Storm,Meteorological,2,83.0,NaN,NaN,NaN,NaN,10000000.0
3093,2018,VNM,Viet Nam,Asia,South-Eastern Asia,Tropical cyclone,Storm,Meteorological,3,48.0,NaN,146000.0,NaN,146000.0,222000.0


In [17]:
df_Americas_woe = pd.DataFrame()
df_Oceania_woe = pd.DataFrame()
df_Africa_woe = pd.DataFrame()


for risk_pool in pools:
    if 'Anguilla' in risk_pool:
        df_time_temp = df_woe[df_woe['year']>=Americas_est]
        for country in risk_pool:
            df_temp = df_time_temp[df_time_temp['country_name']==country]
            df_Americas_woe = df_Americas_woe.append(df_temp)
    if 'Tonga' in risk_pool:
        df_time_temp = df_woe[df_woe['year']>=Oceania_est]
        for country in risk_pool:
            df_temp = df_time_temp[df_time_temp['country_name']==country]
            df_Oceania_woe = df_Oceania_woe.append(df_temp)
    if 'Malawi' in risk_pool:
        df_time_temp = df_woe[df_woe['year']>=Africa_est]
        for country in risk_pool:
            df_temp = df_time_temp[df_time_temp['country_name']==country]
            df_Africa_woe = df_Africa_woe.append(df_temp)            
            
risk_pool_woe_dfs = pd.concat([df_Americas_woe,df_Oceania_woe,df_Africa_woe])
risk_pool_woe_dfs

,year,iso,country_name,continent,region,disaster_subtype,disaster_type,disaster_subgroup,occurrence,Total_deaths,Injured,Affected,Homeless,Total_affected,Total_damage_K_USD
3029,2017,AIA,Anguilla,Americas,Caribbean,Tropical cyclone,Storm,Meteorological,1,4.0,NaN,15000.0,NaN,15000.0,200000.0
253,2010,BRB,Barbados,Americas,Caribbean,Drought,Drought,Climatological,1,NaN,NaN,NaN,NaN,NaN,NaN
2876,2010,BRB,Barbados,Americas,Caribbean,Tropical cyclone,Storm,Meteorological,1,NaN,NaN,2500.0,NaN,2500.0,NaN
3034,2017,BRB,Barbados,Americas,Caribbean,Tropical cyclone,Storm,Meteorological,1,1.0,NaN,NaN,NaN,NaN,NaN
1952,2008,BLZ,Belize,Americas,Central America,Riverine flood,Flood,Hydrological,1,1.0,NaN,38000.0,NaN,38000.0,9697.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
836,2015,KEN,Kenya,Africa,Eastern Africa,Flash flood,Flood,Hydrological,1,13.0,NaN,1500.0,NaN,1500.0,NaN
2477,2015,KEN,Kenya,Africa,Eastern Africa,Riverine flood,Flood,Hydrological,2,125.0,73.0,243726.0,NaN,243799.0,NaN
2515,2016,KEN,Kenya,Africa,Eastern Africa,Riverine flood,Flood,Hydrological,1,3.0,NaN,NaN,1000.0,1000.0,NaN
2539,2017,KEN,Kenya,Africa,Eastern Africa,Riverine flood,Flood,Hydrological,1,26.0,NaN,25000.0,NaN,25000.0,NaN


In [18]:
px.histogram(risk_pool_woe_dfs, y='Affected',x='continent',title='Excluding earthquakes')

In [19]:
px.histogram(risk_pool_woe_dfs, y='occurrence',x='continent',title='Excluding earthquakes')

In [20]:
px.histogram(risk_pool_woe_dfs, y='Total_damage_K_USD',x='continent',title='Excluding earthquakes')